In [1]:
import os, sys
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

Investigate the performance of flights over time or simply look at data for a given year and create a graphic that showcases your finding(s).

In [2]:
def addDate(df):
    import datetime 
    #df = df.head(100000)

    dt = df[['Year', 'Month', 'DayofMonth']]
    dt.columns = ["year", "month", "day"]
    dt.head(2)

    #pd.to_datetime(dt["year", "month", "day"])
    df["Date"] = pd.to_datetime(dt.year*10000 + dt.month*100 + dt.day, format='%Y%m%d')
    return df

In [3]:
## From http://www.transtats.bts.gov/ot_delay/OT_DelayCause1.asp?pn=1
## Download file: http://www.transtats.bts.gov/ot_delay/ot_delaycause1.asp?display=download&pn=0&month=6&year=2016

In [4]:
df_delays = pd.read_csv("./rawData/airline_delay_causes.csv")
print(" Dataframe with {0} registers and the folloging data columns: \n {1}".format(
        len(df_delays), ", ".join(df_delays.columns)))

 Dataframe with 55342 registers and the folloging data columns: 
 year,  month, carrier, carrier_name, airport, airport_name, arr_flights, arr_del15, carrier_ct,  weather_ct, nas_ct, security_ct, late_aircraft_ct, arr_cancelled, arr_diverted,  arr_delay,  carrier_delay, weather_delay, nas_delay, security_delay, late_aircraft_delay, Unnamed: 21


In [5]:
## Download yearly files
## http://stat-computing.org/dataexpo/2009/1992.csv.bz2

http://stat-computing.org/dataexpo/2009/the-data.html

In [9]:
df_origs = {}

In [10]:
#years = [1990,1991,1992,1993]
years = range(1997,2009)
print (" Reading ", years[0])

for kyear in years:
    print (" Reading ", kyear)
    if not kyear in df_origs.keys():
        print (" Here ", kyear)
        try:
            print("Try")
            df_origs[kyear] = pd.read_csv("./rawData/{0}.csv.bz2".format(kyear), engine="c")
        except:
            pass

 Reading  1997
 Reading  1997
 Here  1997
Try
 Reading  1998
 Here  1998
Try
 Reading  1999
 Here  1999
Try
 Reading  2000
 Here  2000
Try
 Reading  2001
 Here  2001
Try
 Reading  2002
 Here  2002
Try
 Reading  2003
 Here  2003
Try


/Users/alberto/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


 Reading  2004
 Here  2004
Try
 Reading  2005
 Here  2005
Try
 Reading  2006
 Here  2006
Try
 Reading  2007
 Here  2007
Try
 Reading  2008
 Here  2008
Try


In [11]:
df_origs[2002] = pd.read_csv("./rawData/{0}.csv".format("2002"), engine="c")

In [12]:
df_origs.keys()
sorted(df_origs.keys())

[1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008]

In [13]:
yk = list(df_origs.keys())
print (yk)
df_orig = df_origs[yk[0]]
#df_orig = pd.concat(df_orig,])

df_orig = addDate(df_orig)

print(" Dataframe with {0} registers and the folloging data columns: \n {1}".format(
        len(df_orig), ", ".join(df_orig.columns)))

[1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008]
 Dataframe with 5411843 registers and the folloging data columns: 
 Year, Month, DayofMonth, DayOfWeek, DepTime, CRSDepTime, ArrTime, CRSArrTime, UniqueCarrier, FlightNum, TailNum, ActualElapsedTime, CRSElapsedTime, AirTime, ArrDelay, DepDelay, Origin, Dest, Distance, TaxiIn, TaxiOut, Cancelled, CancellationCode, Diverted, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, Date


In [14]:
# Number of flights per year

In [55]:
dd = {}
for kyear in df_origs.keys():
    value = len(df_origs[kyear])
    #print(kyear, value)
    dd[kyear] = value

df = pd.DataFrame(dd, index=[0]).T.reset_index() #, orient='index')
df.columns = ["Year", "Value"]
df.set_index("Year", inplace=True)
df.to_json("data/01-Year_NFlights.json")
print( df )

        Value
Year         
1997  5411843
1998  5384721
1999  5527884
2000  5683047
2001  5967780
2002  5271359
2003  6488540
2004  7129270
2005  7140596
2006  7141922
2007  7453215
2008  7009728


# Carriers names

In [48]:
Carriers = set(df_orig["UniqueCarrier"].values.tolist())
print(" Number of carriers {0} (Code):\n\t{1}".format(len(Carriers), ", ".join(Carriers)))

 Number of carriers 19 (Code):
	TZ, MQ, B6, HA, DL, DH, UA, AA, AS, EV, FL, WN, HP, CO, NW, OO, US, XE, OH


In [49]:
def genNamesDict(df_delays):
    cD = df_delays[["carrier","carrier_name"]].drop_duplicates()
    return cD.set_index("carrier").to_dict()["carrier_name"]

cd = genNamesDict(df_delays)
cd = [cd[i] for i in Carriers]
print(" Number of carriers {0} (Names):\n\t{1}".format(len(Carriers), ",\n\t".join(cd)))

 Number of carriers 19 (Names):
	ATA Airlines d/b/a ATA,
	Envoy Air,
	JetBlue Airways,
	Hawaiian Airlines Inc.,
	Delta Air Lines Inc.,
	Independence Air,
	United Air Lines Inc.,
	American Airlines Inc.,
	Alaska Airlines Inc.,
	ExpressJet Airlines Inc.,
	AirTran Airways Corporation,
	Southwest Airlines Co.,
	America West Airlines Inc.,
	Continental Air Lines Inc.,
	Northwest Airlines Inc.,
	SkyWest Airlines Inc.,
	US Airways Inc.,
	ExpressJet Airlines Inc.,
	Comair Inc.


# Data

In [50]:
def defineDataFrame(dff, selK, keep=[]):
    
    dfw = dff.copy()
    print(dfw.columns)
    if not keep == []:
        # Keep indexes
        keep = keep + selK 
        for kcol in dfw.columns:
            if kcol not in keep:
                #print("Removing ", kcol)
                dfw.drop(kcol, inplace=True, axis=1)
    print("Counting")
    grBCou = dfw.groupby(by=selK).count() #.unstack()
    print("Summing")
    grBSum = dfw.groupby(by=selK).sum()
    print("Averaging")
    grBAvg = dfw.groupby(by=selK).mean()
    #grBCou.reset_index()

    dfr = grBAvg.join(
            grBSum, how="outer", 
            rsuffix="_sum").join(
            grBCou, how="outer", 
            lsuffix="_avg", rsuffix="_count").reset_index()

    #print(dfr.columns)
    return dfr

In [51]:
df = df_orig.copy()

In [52]:
df.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'Date'],
      dtype='object')

In [74]:
kl = [ i for i in df.columns if "Delay" in i ]
print(kl)
dfAApre = df.head(8000)[["Origin", "Dest", "Cancelled", "ActualElapsedTime", "CRSElapsedTime"]+kl]
dfAApre.head(8)

['ArrDelay', 'DepDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']


,Origin,Dest,Cancelled,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,ORD,CLT,0,98.0,105,-14.0,-7.0,0,0,0,0,0
1,ORD,CLT,0,110.0,105,-4.0,-9.0,0,0,0,0,0
2,ORD,CLT,0,107.0,105,5.0,3.0,0,0,0,0,0
3,ORD,CLT,0,92.0,105,-16.0,-3.0,0,0,0,0,0
4,ORD,CLT,0,103.0,105,3.0,5.0,0,0,0,0,0
5,ORD,CLT,0,97.0,105,-10.0,-2.0,0,0,0,0,0
6,ORD,CLT,0,114.0,105,29.0,20.0,20,0,9,0,0
7,ORD,CLT,0,97.0,105,-11.0,-3.0,0,0,0,0,0


## Data:: Airport to airport Flights

In [80]:
selK = ["Origin", "Dest"]
delays = [kl]
grBAvg = dfAApre.groupby(by=selK).mean()
grBAvg.reset_index().to_csv("data/Flights-OrigDest-Mean.csv")

dfAA = grBAvg.reset_index()
#dfAA = defineDataFrame(df, selK, df.columns)

In [93]:
Airports = (set(list(dfAA["Origin"].values)+list(dfAA["Dest"].values)))
", ".join(Airports)

'IAH, MHT, BOI, OAK, CLT, TUS, ALB, GEG, ICT, BTV, BUF, SNA, OMA, DTW, MSY, STL, ATL, EWR, IND, BWI, DCA, SLC, JAC, MDT, LAX, FLL, PBI, SAT, TPA, CVG, IAD, LAS, MCI, ABQ, ONT, DEN, BOS, MSP, TUL, CMH, OKC, PHL, BDL, ORD, DSM, RIC, SEA, GSO, PDX, MIA, DFW, PVD, BIL, BUR, COS, MCO, PIT, LGA, SJC, SMF, CLE, PHX, GRR, SAN, SFO'

In [94]:
dfAA.to_csv("data/fligths-DateCancelled.csv")


In [155]:
dfAAA = dfAA.set_index(["Origin","Dest"])
dd = {}
for i,j in dfAAA.index:
    #print(i, j)
    
    try:
        dd[i]["imports"] = dd[i]["imports"] + [j]
        dd[i]["dada"] = dd[i]["dada"] + [j]
    except:
        dd[i] = {} 
        dd[i]["imports"] = [j]
        dd[i]["dada"] = [j]
dd 

{'ABQ': {'dada': ['DEN'], 'imports': ['DEN']},
 'ALB': {'dada': ['ORD'], 'imports': ['ORD']},
 'ATL': {'dada': ['DEN', 'ORD'], 'imports': ['DEN', 'ORD']},
 'BDL': {'dada': ['IAD'], 'imports': ['IAD']},
 'BIL': {'dada': ['DEN'], 'imports': ['DEN']},
 'BOI': {'dada': ['DEN', 'ORD'], 'imports': ['DEN', 'ORD']},
 'BOS': {'dada': ['DEN', 'IAD', 'ORD'], 'imports': ['DEN', 'IAD', 'ORD']},
 'BUR': {'dada': ['DEN'], 'imports': ['DEN']},
 'BWI': {'dada': ['DEN', 'ORD'], 'imports': ['DEN', 'ORD']},
 'CLE': {'dada': ['ORD'], 'imports': ['ORD']},
 'CLT': {'dada': ['DEN', 'ORD'], 'imports': ['DEN', 'ORD']},
 'CMH': {'dada': ['DEN', 'ORD'], 'imports': ['DEN', 'ORD']},
 'DCA': {'dada': ['ORD'], 'imports': ['ORD']},
 'DEN': {'dada': ['ATL',
   'BOI',
   'CLE',
   'CLT',
   'CMH',
   'COS',
   'DFW',
   'DSM',
   'DTW',
   'GEG',
   'IAD',
   'IAH',
   'ICT',
   'IND',
   'JAC',
   'LAS',
   'LAX',
   'MCI',
   'MIA',
   'MSP',
   'OAK',
   'OMA',
   'ONT',
   'ORD',
   'PDX',
   'PHL',
   'PHX',
   'SE

In [166]:
dfAAAA = pd.DataFrame(dd).T
dfAAAA.index.name = "name"
dfAAAA.head(2)

,dada,imports
name,,
ABQ,[DEN],[DEN]
ALB,[ORD],[ORD]


In [170]:
dfAAAA.reset_index().to_json("data/Airports-flare.json", orient="records")

In [176]:
dfAAA.reset_index().to_json("data/Airports-flare2.json", orient="records")

In [123]:
for i, j in dfAAA.index:
    idx = (i,j)
    print(idx)
    print(dfAAA.loc[idx])

('ABQ', 'DEN')
Cancelled             0.00
ActualElapsedTime    75.48
CRSElapsedTime       76.00
ArrDelay             -1.24
DepDelay             -0.72
CarrierDelay          3.44
WeatherDelay          0.00
NASDelay              0.40
SecurityDelay         0.00
LateAircraftDelay     0.00
Name: (ABQ, DEN), dtype: float64
('ALB', 'ORD')
Cancelled              0.000000
ActualElapsedTime    165.333333
CRSElapsedTime       136.000000
ArrDelay              94.833333
DepDelay              65.500000
CarrierDelay           0.000000
WeatherDelay           0.000000
NASDelay              57.333333
SecurityDelay          0.000000
LateAircraftDelay     37.500000
Name: (ALB, ORD), dtype: float64
('ATL', 'DEN')
Cancelled              0.000000
ActualElapsedTime    198.612903
CRSElapsedTime       197.806452
ArrDelay               7.516129
DepDelay               6.709677
CarrierDelay           5.693548
WeatherDelay           0.000000
NASDelay               2.629032
SecurityDelay          0.000000
LateAircraf

## Data:: Date vs Cancelled Flights

In [ ]:
selK = ["Date"]
delays = ["Cancelled"]
df1 = defineDataFrame(df, selK, delays)

df1.head(2)

In [ ]:
### Date vs mean delay 
df1.set_index("Date").to_csv("data/fligths-DateCancelled.csv")

## Data:: Date vs Delayed Flights

In [ ]:
selK = ["Date"]
delays = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
df2 = defineDataFrame(df, selK, delays)

df2.head(2)

In [ ]:
df2.set_index("Date").to_csv("data/fligths-DateDelayed.csv")

## Data:: Year vs Delayed Flights

In [ ]:
selK = ["Year"]
delays = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
df3 = defineDataFrame(df, selK, delays)

df3.head(2)

In [ ]:
df3.set_index(selK).to_csv("data/fligths-YearDelayed.csv")

In [ ]:
## Data:: Year vs Delayed Flights

In [ ]:
selK = ["year"]
delays = ["arr_delay",  "carrier_delay", "weather_delay", "nas_delay", "security_delay", "late_aircraft_delay"]

In [ ]:
df_delays.columns 

In [ ]:
df4 = defineDataFrame(df_delays, selK, delays)
df4.head(2)

In [ ]:
df4.set_index(selK).to_csv("data/flights-YearDelayed.csv")